In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import cv2
import os
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

def load_and_preprocess_data(data_dir):
    images = []
    labels = []
    
    image_dir = os.path.join(data_dir, 'images')
    label_dir = os.path.join(data_dir, 'labels')
    
    for image_name in os.listdir(image_dir):
        # Load and preprocess image
        image_path = os.path.join(image_dir, image_name)
        image = cv2.imread(image_path)
        image = cv2.resize(image, (224, 224))
        image = image / 255.0
        images.append(image)
        
        # Load corresponding label
        label_name = os.path.splitext(image_name)[0] + '.txt'
        label_path = os.path.join(label_dir, label_name)
        
        with open(label_path, 'r') as f:
            label_lines = f.readlines()
            # Extract class IDs from each line
            class_ids = [int(line.split()[0]) for line in label_lines]
            
            # For binary classification, we'll consider an image positive if it contains any drone
            label = 1 if 1 in class_ids else 0
        
        labels.append(label)
    
    return np.array(images), np.array(labels)

# Load and preprocess the data
data_dir = r"C:\Users\Intern13\Desktop\project\qtm\dataset\yolo_dataset_v3_negative\train"
X, y = load_and_preprocess_data(data_dir)
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
def create_model():
    inputs = Input(shape=(224, 224, 3))
    
    x = Conv2D(32, (3, 3), activation='relu')(inputs)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = Flatten()(x)
    x = Dense(64, activation='relu')(x)
    outputs = Dense(1, activation='sigmoid')(x)  # Changed to sigmoid for binary classification
    
    model = Model(inputs=inputs, outputs=outputs)
    return model

# Create the model
model = create_model()

# Step 3: Model Compilation
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])  # Changed to binary_crossentropy

# Add early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Step 4: Model Training
# Train the model
history = model.fit(
    X_train, y_train,
    epochs=5,
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping])


# Step 5: Model Evonooo
test_loss, test_acc = model.evaluate(X_val, y_val, verbose=2)
print(f"Test accuracy: {test_acc}")
# Step 6: Convert model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TFLite model
with open('drone_detection_model.tflite', 'wb') as f:
    f.write(tflite_model)
# Step 7: Quantize the model (optional, for better performance on STM board)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_tflite_model = converter.convert()

# Save the quantized TFLite model
with open('drone_detection_model_quantized.tflite', 'wb') as f:
    f.write(quantized_tflite_model)    

# Step 8: Test the TFLite model
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

def predict_tflite(image):
    input_data = np.expand_dims(image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    return output_data[0][0]

# Test the TFLite model on a sample image
sample_image = X_val[0]
prediction = predict_tflite(sample_image)
print(f"TFLite model prediction: {prediction}")

In [ ]:
# Plot training history


plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()